In [ ]:
import cv2
import cvlib as cv
from cvlib.object_detection import draw_bbox
import socket
import queue
import threading


thread_stopper = threading.Event()
ir_led_state = False
connected = False
payload_queue = queue.Queue()
tracker_queue = queue.Queue()
client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)


while not connected :
    try:
        server_port = int(input('Enter Server Port Number:'))
        server_ip_address = input('Enter Server Ip Address:')
        client_socket.connect((server_ip_address,server_port))
        connected = True
    except:
        continue
        
secret_certificate_key = input('Enter Server Certificate Key:')
           
def receive():
    global thread_stopper
    while not thread_stopper.is_set():
        response = client_socket.recv(1024).decode()
        print(response)
        
def transmit():
    global thread_stopper
    while not thread_stopper.is_set():
        if not payload_queue.empty():
            client_socket.send(payload_queue.get())
    
    
def capture():
    global ir_led_state
    
    video_capture = cv2.VideoCapture(0)
    while video_capture.isOpened():  
        ret, frame = video_capture.read()
        tracker_queue.put(frame,block = False)
        bbox,label,conf =cv.detect_common_objects(frame)
        
        for i in range(len(label)):
            if label[i] == 'cell phone':
                x, y, w, h = bbox[i]
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, 'cell phone', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 255), 2)
        
        cv2.imshow('tracked', frame)
        if 'cell phone' in label and ir_led_state == False:
            print('Cellphone Detected')
            ir_led_state = True
            payload = secret_certificate_key +':'+'ON'
            payload_queue.put(payload.encode())
        elif 'cell phone' not in label and ir_led_state == True:
            ir_led_state = False
            payload = secret_certificate_key +':'+'OFF'
            payload_queue.put(payload.encode())
        if cv2.waitKey(1) & 0xFF == ord('q'):
            thread_stopper.set()
            break 
      
    video_capture.release()
    cv2.destroyAllWindows()
   


 
receiver_thread = threading.Thread(target=receive)
transmitter_thread = threading.Thread(target=transmit)



transmitter_thread.start()
receiver_thread.start()

capture()

transmitter_thread.join()
receiver_thread.join()

client_socket.close()


Enter Server Port Number:10203
Enter Server Ip Address:192.168.29.35
Enter Server Certificate Key:!@#$%^&*()_+
Cellphone Detected
Cellphone Detected
Cellphone Detected
Cellphone Detected
Cellphone Detected
